In [3]:
import chess.pgn
from IPython.display import clear_output
import chess.svg

pgn = open("DATABASE4U.pgn", encoding="utf-8")
first_game = chess.pgn.read_game(pgn)

In [4]:
# Connect to MongoDB
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
mongodb = client.chessDB
gameCollection = mongodb.gameCollection
playerCollection = mongodb.playerCollection

In [5]:
def getHeaderData(header):
    try:
        event = header['Event']
        date = header['Date'] if 'Date' in header and '?' not in header['Date'] else 1/0
        white = header['White']
        black = header['Black']
        result = header['Result']
        eco = header['ECO']
        blackElo = header['BlackElo'] if not header['BlackElo'] == "NA" else 1/0
        whiteElo = header['WhiteElo'] if not header['WhiteElo'] == "NA" else 1/0
    except:
        return False
    
    newHeader = {}
    newHeader['Event'] = event
    newHeader['Date'] = date
    newHeader['White'] = white
    newHeader['Black'] = black
    newHeader['Result'] = result
    newHeader['ECO'] = eco
    newHeader['BlackElo'] = blackElo
    newHeader['WhiteElo'] = whiteElo

    return newHeader

In [2]:
import pprint

# Define a nested dictionary
data = {
    'foo': 123,
    'bar': {
        'baz': [1, 2, 3],
        'qux': {
            'spam': 'ham',
            'eggs': 'bacon'
        }
    }
}

# Use pprint to print the dictionary with a 4-space indent
pprint.pprint(data, indent=4)


{'bar': {'baz': [1, 2, 3], 'qux': {'eggs': 'bacon', 'spam': 'ham'}}, 'foo': 123}


In [6]:
# Open PGN file and read games
with open("DATABASE4U.pgn", encoding="latin-1") as pgn_file:
    i = 500
    while True:
        game = chess.pgn.read_game(pgn_file)
        if game is None:
            break
        headers = getHeaderData(game.headers)
        if not headers:
            continue
        moves = [str(move) for move in game.mainline_moves()]
        game_data = {"headers": headers, "moves": moves}
        gameCollection.insert_one(game_data)
        if i == 0:
            break
        i -= 1

<Game at 0x214c958e0e0 ('Heinemann, T.' vs. 'Krauss, Jo', '31.12.29')>
{   'headers': {   'Black': 'Krauss, Jo',
                   'BlackElo': '2154',
                   'Date': '31.12.29',
                   'ECO': 'B07',
                   'Event': '12th Forchheim Open A 2013',
                   'Result': '1/2-1/2',
                   'White': 'Heinemann, T.',
                   'WhiteElo': '2461'},
    'moves': [   'e2e4',
                 'd7d6',
                 'd2d4',
                 'g8f6',
                 'b1c3',
                 'g7g6',
                 'c1e3',
                 'c7c6',
                 'd1d2',
                 'b7b5',
                 'f2f3',
                 'b8d7',
                 'g1h3',
                 'c8b7',
                 'f1e2',
                 'd8c7',
                 'h3f2',
                 'e7e5',
                 'e1g1',
                 'f8g7',
                 'd4e5',
                 'd6e5',
                 'a2a4',
                 '

In [10]:
# Get all documents in the collection
gameDocuments = gameCollection.find()
playerDocuments = playerCollection.find()
playerDataAll = {}

allNames = []
for doc in playerDocuments:
    try:
        allNames.append(doc['name'])
    except:
        print(doc)

# Print each document
for doc in gameDocuments:
    nameWhite = doc['headers']['White']
    ratingWhite = doc['headers']['WhiteElo']
    nameBlack = doc['headers']['Black']
    ratingBlack = doc['headers']['BlackElo']
    date = doc['headers']['Date']
    result = doc['headers']['Result']
    win, loss = result.split("-")

    if nameWhite in allNames:
        # update wins losses or draws
        myquery = {"name": nameWhite}
        myPlayer = playerCollection.find_one(myquery)
        if myPlayer is None:
            continue
        if win == '1':
            numWins = myPlayer['wins']
            numWins += 1
            newvalues = { "$set": { "wins": numWins } }
        if win == '0':
            numLosses = myPlayer['losses']
            numLosses += 1
            newvalues = { "$set": { "losses": numLosses } }
        if win == '1/2' or win == '0.5':
            numDraws = myPlayer['losses']
            numDraws += 1
            newvalues = { "$set": { "draws": numDraws } }
        playerCollection.update_one(myquery, newvalues)

        #update date
        newDate = myPlayer['dates']
        newDate.append(date)
        newvalues = { "$set": { "dates": newDate } }
        playerCollection.update_one(myquery, newvalues)

        #update rating
        newRating = myPlayer['ratings']
        newRating.append(ratingWhite)
        newvalues = { "$set": { "ratings": newRating} }
        playerCollection.update_one(myquery, newvalues)

    else:
        # create document
        if win == '1':
            playerData = {'name': nameWhite,
                          'wins': 1,
                          'losses': 0,
                          'draws': 0,
                          'dates':[date],
                          'ratings':[ratingWhite]
                          }
        if win == '0':
            playerData = {'name': nameWhite,
                          'wins': 0,
                          'losses': 1,
                          'draws': 0,
                          'dates':[date],
                          'ratings':[ratingWhite]
                          }
        if win == '1/2' or win == '0.5':
            playerData = {'name': nameWhite,
                          'wins': 0,
                          'losses': 0,
                          'draws': 1,
                          'dates':[date],
                          'ratings':[ratingWhite]
                          }
        playerCollection.insert_one(playerData)
        allNames.append(nameWhite)


    if nameBlack in allNames:
        # update wins losses or draws
        myquery = {"name": nameBlack}
        myPlayer = playerCollection.find_one(myquery)
        if myPlayer is None:
            continue
        if loss == '1':
            numWins = myPlayer['wins']
            numWins += 1
            newvalues = { "$set": { "wins": numWins } }
        if loss == '0':
            numLosses = myPlayer['losses']
            numLosses += 1
            newvalues = { "$set": { "losses": numLosses } }
        if loss == '1/2' or loss == '0.5':
            numDraws = myPlayer['draws']
            numDraws += 1
            newvalues = { "$set": { "draws": numDraws } }
        playerCollection.update_one(myquery, newvalues)

        #update date
        newDate = myPlayer['dates']
        newDate.append(date)
        newvalues = { "$set": { "dates": newDate} }
        playerCollection.update_one(myquery, newvalues)

        #update rating
        newRating = myPlayer['ratings']
        newRating.append(ratingBlack)
        newvalues = { "$set": { "ratings": newRating} } 
        playerCollection.update_one(myquery, newvalues)
    else:
        # create document
        if loss == '1':
            playerData = {'name': nameBlack,
                          'wins': 1,
                          'losses': 0,
                          'draws': 0,
                          'dates':[date],
                          'ratings':[ratingBlack]
                          }
        if loss == '0':
            playerData = {'name': nameBlack,
                          'wins': 0,
                          'losses': 1,
                          'draws': 0,
                          'dates':[date],
                          'ratings':[ratingBlack]
                          }
        if loss == '1/2' or loss == '0.5':
            playerData = {'name': nameBlack,
                          'wins': 0,
                          'losses': 0,
                          'draws': 1,
                          'dates':[date],
                          'ratings':[ratingBlack]
                          }
        playerCollection.insert_one(playerData)
        allNames.append(nameBlack)